In [1]:
%matplotlib inline
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import math
import time
import numpy as np
import tensorflow as tf
from d2l import tensorflow as d2l


# 1.图计算
0. eager execution即刻计算：  
由python解释器执行python代码
1. graph execution图计算：  
构建计算图，